In [1]:
import pandas as pd
import numpy as np

In [2]:
# File location on S3 (The address should be updated depending on the version of the code using)
loc_2018 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220306/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_tripid = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220316/beam/year-2018-iteration-2/ITERS/it.0/"
loc_2018_tripid_modified = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220327/beam/year-2018-iteration-5/ITERS/it.0/"

In [3]:
%%time
# Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "reason": "category",
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object"
}

# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(loc_2018_tripid_modified + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:21: DtypeWarning: Columns (5,7,12,13,14,15,24,30,33,50,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 3min 28s


In [5]:
#plansSF = pd.read_csv(loc_2018_tripid + '0.plans.csv.gz', compression = 'gzip', dtype = dtypes)

In [6]:
#plansSF.sort_values(by = "personId").head()

In [4]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [6]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [7]:
eventsSFCopy = eventsSF.copy()

In [8]:
# shift column 'Name' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [9]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [10]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [11]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [12]:
# Shift column 'Name' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [13]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

Wall time: 1min 41s


In [14]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [15]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [16]:
# Either charging events or just ridehail or transit drivers
IDnan = eventsSF[eventsSF['IDMerged'].isna()]

In [17]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [18]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(3442, 63)

In [19]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

Wall time: 4min 27s


In [20]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [21]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [22]:
print(eventsSF.currentTourMode.unique().tolist())

['car', 'hov2_teleportation', nan, 'car_hov3', 'bike', 'walk', 'hov3_teleportation', 'car_hov2', 'walk_transit', 'ride_hail', 'drive_transit', 'ride_hail_pooled', 'ride_hail_transit', 'bike_transit']


In [23]:
print(eventsSF.modeBEAM.unique().tolist())

['car', 'hov2_teleportation', nan, 'walk', 'car_hov3', 'bike', 'hov3_teleportation', 'car_hov2', 'walk_transit', 'ride_hail', 'drive_transit', 'ride_hail_pooled', 'ride_hail_transit', 'bus', 'tram', 'subway', 'rail', 'cable_car', 'bike_transit']


In [24]:
print(eventsSF.type.unique().tolist())

['ModeChoice', 'actend', 'departure', 'PersonEntersVehicle', 'TeleportationEvent', 'arrival', 'actstart', 'PathTraversal', 'LeavingParkingEvent', 'ParkingEvent', 'PersonCost', 'PersonLeavesVehicle', 'ReserveRideHail', 'ChargingPlugInEvent', 'RefuelSessionEvent', 'ChargingPlugOutEvent', 'Replanning']


In [25]:
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

In [26]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [27]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ridehail' , eventsSF['modeBEAM'])

In [28]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [29]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [44]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [30]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [46]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [31]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'].notna()), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [48]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'].notna()), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [51]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

In [50]:
#eventsSF[eventsSF['IDMerged'] == 11024]

In [38]:
dist = eventsSF.head(55000)

In [39]:
dist.to_csv('C:/Shared-Work/Data/CleanData/dist.csv', index = False)

In [56]:
#eventsSF[eventsSF['modeBEAM'] == "ride_hail_pooled"].head()

#### Adding new columns

In [52]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [53]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [54]:
eventsSF['travelTime'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [55]:
eventsSF['travelDistance'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [56]:
eventsSF['length_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [57]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['travelTime'], np.nan)

In [58]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['travelDistance'], np.nan)

In [59]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['travelTime'], np.nan)

In [60]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['travelDistance'], np.nan)

In [61]:
eventsSF['duration_in_ridehail'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail', eventsSF['travelTime'], np.nan)

In [62]:
eventsSF['distance_ridehail'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail', eventsSF['travelDistance'], np.nan)

In [63]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['travelTime'], np.nan)

In [64]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['travelDistance'], np.nan)

In [65]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['travelTime'], np.nan)

In [66]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['travelDistance'], np.nan)

In [67]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [55]:
eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [52]:
eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                eventsSF['primaryFuel'], np.nan)

In [53]:
eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [54]:
eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [55]:
eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            , eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [56]:
eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [68]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [69]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

In [58]:
#eventsSF[(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled')|(eventsSF['type'] == 'Replanning')].head()

In [72]:
eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripIndex.fillna(method='ffill')

In [73]:
# Make a new column which determined mode choice numbers for each person and trip
s = eventsSF.groupby(['IDMerged','tripIndex', 'type']).cumcount().add(1).astype(str).str.zfill(2)

In [74]:
eventsSF['type_number'] = eventsSF['type'].astype(str) + s.astype(str)

In [75]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice01' , eventsSF['modeBEAM_rh'], np.nan)

In [76]:
eventsSF['mode_choice_replanned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice02', eventsSF['modeBEAM_rh'], np.nan)

In [79]:
eventsSF[eventsSF['type'] == "Replanning"].head()

,IDMerged,person,driver,riders,vehicle,time,type,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,price,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,endY,endX,startY,startX,arrivalTime,departureTime,secondaryFuelLevel,primaryFuelLevel,cost,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,links,numPassengers,primaryFuel,departTime,score,incentive,tollCost,netCost,modeBEAM_rh,actEndTime,actStartTime,travelTime,travelDistance,length_mode_choice,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,actEndType,actStartType,tripIndex,type_number,mode_choice_planned_BEAM,mode_choice_replanned_BEAM
519,109,109,NaN,NaN,NaN,42447.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,2.0,Replanning01,NaN,NaN
522,109,109,NaN,NaN,NaN,42453.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,2.0,Replanning02,NaN,NaN
525,109,109,NaN,NaN,NaN,42459.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,2.0,Replanning03,NaN,NaN
1299,319,319,NaN,NaN,NaN,30720.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,Replanning01,NaN,NaN
1302,319,319,NaN,NaN,NaN,30976.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,Replanning02,NaN,NaN


In [80]:
eventsSF[eventsSF['IDMerged']==109]

,IDMerged,person,driver,riders,vehicle,time,type,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,price,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,endY,endX,startY,startX,arrivalTime,departureTime,secondaryFuelLevel,primaryFuelLevel,cost,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,links,numPassengers,primaryFuel,departTime,score,incentive,tollCost,netCost,modeBEAM_rh,actEndTime,actStartTime,travelTime,travelDistance,length_mode_choice,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,actEndType,actStartType,tripIndex,type_number,mode_choice_planned_BEAM,mode_choice_replanned_BEAM
485,109,109,NaN,NaN,NaN,33224.0,ModeChoice,walk_transit,walk_transit,NaN,WALK_TRANSIT,134154.0,false,551.723679,1.0,"WALK,BUS,WALK","body-109,SF:7614725,body-109",Home,work,36065.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,walk_transit,NaN,NaN,NaN,NaN,551.723679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,,,1.0,ModeChoice01,walk_transit,NaN
486,109,109,NaN,NaN,NaN,33224.0,actend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134154.0,NaN,NaN,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33224.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home,,1.0,actend01,NaN,NaN
487,109,109,NaN,NaN,NaN,33224.0,departure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134154.0,walk_transit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,departure01,NaN,NaN
488,109,109,NaN,NaN,body-109,33224.0,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,PersonEntersVehicle01,NaN,NaN
489,109,NaN,109,109,body-109,33453.0,PathTraversal,walk,walk_transit,NaN,NaN,NaN,NaN,297.783000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BODY-TYPE-DEFAULT,37.782152,-122.447286,37.781835,-122.453101,33453.0,33224.0,0.0,2.194218e+06,NaN,NaN,NaN,0.0,0.0,0.0,"212.572,28.794,153.698,18.803,8.435,8.362,6.03...",0.0,None,Food,"134154,142864,22815,22813,8630,47102,80882,144111",1.0,15782.499,NaN,NaN,NaN,NaN,NaN,walk,NaN,NaN,229.0,297.783,NaN,229.0,297.783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,PathTraversal01,NaN,NaN
490,109,109,NaN,NaN,SF:7614725,34041.0,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,PersonEntersVehicle02,NaN,NaN
491,109,109,NaN,NaN,NaN,34041.0,PersonCost,walk_transit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.75,walk_transit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [66]:
%%time
eventsSF["mode_choice_replanned_BEAM"] = eventsSF.groupby(['IDMerged','tripIndex'])['mode_choice_replanned_BEAM'].transform(lambda x: x.bfill())

KeyboardInterrupt: 

In [66]:
eventsSF['mode_choice_actual_BEAM'] = eventsSF['mode_choice_replanned_BEAM'].combine_first(eventsSF['mode_choice_planned_BEAM'])

In [67]:
eventsSF['mode_choice_actual_BEAM'] = eventsSF['mode_choice_actual_BEAM'].combine_first(eventsSF['modeBEAM_rh'])

In [ ]:
#eventsSF['mode_planned_tour_ActSim'] = np.where(eventsSF['type'] == 'PathTraversal', eventsSF['currentTourMode'], np.nan)

In [87]:
eventsSF.head(10000).to_csv('C:/Shared-Work/Data/CleanData/eventsNew_check.csv', index = False)   

In [98]:
#eventsSF['numPassengers_agg'] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False).numPassengers.agg(lambda x: ', '.join(set(x.dropna().astype(str))))

#### Trip Index

In [59]:
eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripIndex.fillna(method='ffill')

In [ ]:
eventsSF[eventsSF['IDMerged']==181]

In [63]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [104]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'travelTime': np.sum, 'cost_BEAM': np.sum, 'actStartType': np.sum, 
            'actEndType': np.sum, 'duration_walking': np.sum, 'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 'travelDistance': np.sum, 'duration_in_transit': np.sum, 'distance_walking': np.sum, 
            'distance_bike': np.sum, 'distance_ridehail': np.sum, 'distance_privateCar': np.sum, 'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 'fuelFood': np.sum, 'fuelElectricity': np.sum, 'fuelBiodiesel': np.sum, 
            'fuelDiesel': np.sum, 'fuel_not_Food': np.sum, 'fuelGasoline': np.sum, 'mode_choice_planned_BEAM':np.sum, 
            'tripId': np.sum, 'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'length_mode_choice': np.sum}).reset_index()  
#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str)))
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str)))

Wall time: 8min 8s


In [105]:
Person_Trip_eventsSF['door_to_door_time'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [106]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['door_to_door_time'] - Person_Trip_eventsSF['travelTime'] 

In [107]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [108]:
Person_Trip_eventsSF.rename(columns={"modeBEAM_rh":"mode_BEAM_list"}, inplace=True) 

In [115]:
Person_Trip_eventsSF.head()

,IDMerged,tripIndex,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,distance_privateCar,distance_ridehail,distance_transit,distance_walking,duration_in_privateCar,duration_in_ridehail,duration_in_transit,duration_on_bike,duration_walking,fuelBiodiesel,fuelDiesel,fuelElectricity,fuelFood,fuelGasoline,fuel_not_Food,legVehicleIds,length_mode_choice,mode_BEAM_list,mode_choice_planned_BEAM,numPassengers,travelDistance,travelTime,tripId,door_to_door_time,waitTime,actPurpose
0,1,1.0,29142.0,Home,29396.0,othmaint,0.000000,0.0,4021.422,0.0,0.0,0.000,254.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000e+00,0.000000e+00,"body-1,teleportationSharedVehicle-1-1,body-1",4021.422,"hov3_teleportation, hov3_teleportation",hov3_teleportation,,4021.422,254.0,577.0,254.0,0.0,Home_to_othmaint
1,1,2.0,31741.0,othmaint,31867.0,school,0.000000,0.0,2169.162,0.0,0.0,0.000,126.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000e+00,0.000000e+00,"body-1,teleportationSharedVehicle-2-1,body-1",2169.162,"hov3_teleportation, hov3_teleportation",hov3_teleportation,,2169.162,126.0,578.0,126.0,0.0,othmaint_to_school
2,1,3.0,52106.0,school,52298.0,Home,0.000000,0.0,2680.151,0.0,0.0,0.000,192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000e+00,0.000000e+00,"body-1,teleportationSharedVehicle-3-1,body-1",2680.151,"hov3_teleportation, hov3_teleportation",hov3_teleportation,,2680.151,192.0,581.0,192.0,0.0,school_to_Home
3,2,1.0,24462.0,Home,25714.0,work,0.161166,0.0,1469.429,0.0,0.0,1469.429,122.0,0.0,0.0,0.0,1130.0,0.0,0.0,0.0,77879.737,2.018397e+06,2.018397e+06,"body-2,539168,539168,body-2",2938.858,"car, walk, car, car, car, walk",car,"1.0, 0.0, 0.0, 1.0",2938.858,1252.0,969.0,1252.0,0.0,Home_to_work
4,2,2.0,44157.0,work,46155.0,atwork,3.757355,0.0,34257.620,0.0,0.0,786.423,1393.0,0.0,0.0,0.0,605.0,0.0,0.0,0.0,41680.419,5.090772e+07,5.090772e+07,"body-2,539168,539168,body-2",35044.043,"car, walk, car, car, car, walk",car,"1.0, 0.0, 0.0, 1.0",35044.043,1998.0,689.0,1998.0,0.0,work_to_atwork


In [89]:
eventsSF[eventsSF['type'] == 'Replanning'].head()

,IDMerged,person,driver,riders,vehicle,time,type,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,price,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,endY,endX,startY,startX,arrivalTime,departureTime,secondaryFuelLevel,primaryFuelLevel,cost,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,links,numPassengers,primaryFuel,departTime,score,incentive,tollCost,cost_BEAM,modeBEAM_rh,actEndTime,actStartTime,travelTime,travelDistance,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,fuelFood,fuelElectricity,fuelDiesel,fuelBiodiesel,fuel_not_Food,fuelGasoline,actEndType,actStartType,tripIndex,type_number,mode_choice_planned_BEAM,mode_choice_replanned_BEAM,mode_choice_actual_BEAM
519,109,109,NaN,NaN,NaN,42447.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,2.0,Replanning01,NaN,walk_transit,walk_transit
522,109,109,NaN,NaN,NaN,42453.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,2.0,Replanning02,NaN,NaN,NaN
525,109,109,NaN,NaN,NaN,42459.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,2.0,Replanning03,NaN,NaN,NaN
1299,319,319,NaN,NaN,NaN,30720.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,Replanning01,NaN,walk_transit,walk_transit
1302,319,319,NaN,NaN,NaN,30976.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,Replanning02,NaN,NaN,NaN


In [111]:
b = Person_Trip_eventsSF[Person_Trip_eventsSF['IDMerged'] == 108]

In [79]:
Person_Trip_eventsSF.head(10000).to_csv('C:/Shared-Work/Data/CleanData/pivot_check_list.csv', index = False)   

In [116]:
sample_bike2 = eventsSF[(eventsSF['primaryFuelType'] == "Gasoline") & (eventsSF['modeBEAM'] == "bike") ]

In [120]:
sample_bike2.head()

,IDMerged,person,driver,riders,vehicleType,price,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicle,actType,time,type,link,legMode,primaryFuelLevel,secondaryFuelLevel,facility,score,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,reason,links,numPassengers,primaryFuel,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,endY,endX,startY,startX,capacity,arrivalTime,departureTime,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,incentive,tollCost,netCost,departTime,cost
248,468282,NaN,468282,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49567,NaN,18146.0,PathTraversal,NaN,NaN,2.203726e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,591.851,NaN,NaN,NaN,NaN,NaN,NaN,"71796,71799,29023,13743,13742",0.0,6273.6206,NaN,NaN,2.0,0.0,37.337513,-122.057953,37.337514,-122.060283,2.0,18146.0,18028.0,"27.254,13.274,0.638,52.229,52.229",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
316,2925369,NaN,2925369,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,734823,NaN,18095.0,PathTraversal,NaN,NaN,2.205912e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,385.622,NaN,NaN,NaN,NaN,NaN,NaN,"10188,10189",0.0,4087.5932,NaN,NaN,2.0,0.0,37.967986,-121.785963,37.967986,-121.785963,2.0,18095.0,18018.0,"77.124,77.124",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
16062,2643355,NaN,2643355,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202653,NaN,18250.0,PathTraversal,NaN,NaN,2.197108e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,1216.258,NaN,NaN,NaN,NaN,NaN,NaN,"68460,82455,69029,69022",0.0,12892.3348,NaN,NaN,2.0,0.0,37.839078,-122.275288,37.828203,-122.275229,2.0,18250.0,18007.0,"67.23,7.505,56.199,179.548",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
18520,2008662,NaN,2008662,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,737344,NaN,18291.0,PathTraversal,NaN,NaN,2.209720e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,26.422,NaN,NaN,NaN,NaN,NaN,NaN,"162585,162584",0.0,280.0732,NaN,NaN,2.0,0.0,37.473750,-122.232240,37.473750,-122.232240,2.0,18291.0,18286.0,"5.284,5.284",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN
26940,3707112,NaN,3707112,NaN,BIKE-DEFAULT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52636,NaN,18416.0,PathTraversal,NaN,NaN,2.206459e+06,0.0,NaN,NaN,bike,bike,NaN,NaN,NaN,NaN,334.028,NaN,NaN,NaN,NaN,NaN,NaN,"35377,35376",0.0,3540.6968,NaN,NaN,2.0,0.0,37.502374,-122.270943,37.502374,-122.270943,2.0,18416.0,18349.0,"66.806,66.806",0.0,None,Gasoline,NaN,NaN,NaN,NaN,NaN


In [112]:
b.to_csv('C:/Shared-Work/Data/CleanData/b.csv', index = False)

#### Merging with activity sim persons and housholds files

In [73]:
actloc_2018 =  "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220327/activitysim/"

households = pd.read_csv(actloc_2018 + 'final_households.csv')
persons = pd.read_csv(actloc_2018 + 'final_persons.csv')
tours = pd.read_csv(actloc_2018 +'final_tours.csv')
plans = pd.read_csv(actloc_2018 +'final_plans.csv')
trips = pd.read_csv(actloc_2018 + 'final_trips.csv')

In [74]:
#Merge households and persons 
persons = persons.sort_values(by=['household_id'])
households = households.sort_values(by=['household_id'])
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [75]:
#Merge tours, households and persons
tours = tours.sort_values(by=['person_id'])
hhpersons = hhpersons.sort_values(by=['person_id'])
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [76]:
#Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id'])
hhperTours = hhperTours.sort_values(by=['person_id','tour_id'])
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

Wall time: 9.8 s


In [77]:
tourTripsMerged.head()

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,tdd,start,end,duration,composition,tour_mode,atwork_subtour_frequency,parent_tour_id,stop_frequency,earning,worker,student,hispanic.1,person_sex,PNUM,sex,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
0,577,1,1735309,72,school,1,True,2,othmaint,583,641,6.776413,8.0,SHARED3FREE,-1.948476,school,1,1,1,1,mandatory,1,61.0,8.0,15.0,7.0,NaN,SHARED3FREE,NaN,NaN,1out_0in,0.0,0,1,0,male,3,1,0.0,5,19 and under,1.0,633,6,asian,0,3,-1,no,2,0,641,8,4,1,-121.80644,37.350304,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,True,True,False,False,1,False,641,19.798008,634,1.505897,21.066668,-1,NaN,NaN,False,NaN,0.000000,0.0000,0.0,False,M,True,False,False,False,school1,False,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2,6085,yes,60855035091006,1,no,35,6,own not recent,four or more,yes,1,two or more,240000.0,gt35-lt65,2010000482865,2.0,asian,gt150,0,two or more,0,no,4,3,4,0.097,60138,240.0,4,12.86,29.682171,2.0,2,2,2,2,0,0,0,1,True,False,False,False,2,1.392788,0,4,2,2,2,2,True,0_tours,0
1,578,1,1735309,72,school,2,True,2,school,634,583,NaN,8.0,SHARED2FREE,-1.628539,school,1,1,1,1,mandatory,1,61.0,8.0,15.0,7.0,NaN,SHARED3FREE,NaN,NaN,1out_0in,0.0,0,1,0,male,3,1,0.0,5,19 and under,1.0,633,6,asian,0,3,-1,no,2,0,641,8,4,1,-121.80644,37.350304,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,True,True,False,False,1,False,641,19.798008,634,1.505897,21.066668,-1,NaN,NaN,False,NaN,0.000000,0.0000,0.0,False,M,True,False,False,False,school1,False,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,2,6085,yes,60855035091006,1,no,35,6,own not recent,four or more,yes,1,two or more,240000.0,gt35-lt65,2010000482865,2.0,asian,gt150,0,two or more,0,no,4,3,4,0.097,60138,240.0,4,12.86,29.682171,2.0,2,2,2,2,0,0,0,1,True,False,False,False,2,1.392788,0,4,2,2,2,2,True,0_tours,0
2,581,1,1735309,72,school,1,False,1,

In [79]:
tourTripsMerged = tourTripsMerged.sort_values(by=['person_id', 'trip_id'])
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripId'])
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

In [82]:
eventsASim.head(10000).to_csv('C:/Shared-Work/Data/CleanData/eventsASim.csv', index = False)   